# Post-processing

Output of model will be segmentation probabilities.

We need to get labeled regions.

Mains steps:

* Binarize
* Label
* Remove small objects
* Close holes inside objects

In [1]:
from skimage.measure import label, regionprops

min_area = 15
def post_process(mask):
    lab = label(mask)
    for obj in regionprops(lab):
        if( obj.area < min_area ):
            lab[lab==obj.label] = 0
        lab[obj.bbox[0]:obj.bbox[2],obj.bbox[1]:obj.bbox[3]] = obj.filled_image*obj.label
    
    return lab